Default agent Orchestrator with multiple handoff tools

Orchestrator:
User Prompt and file path

Tools:
Handoff tool to Pre-processor
Handoff tool to Compliance Checker (Knowledge Agent for now)
Handoff tool to Post-processor

	Prompt: “  “

	Name: Orchestrator


Pre-processor:
Take in a file path as input

Tools:
OCR
NER
Classification and clause extraction
Handoff tool for Orchestrator

Prompt: “  ”

Name: Preprocessor Agent



Knowledge Agent:
Search Query with sitename and keywords

Tools:
Web Search tool
Handoff tool to Compliance (Orchestrator for now)

Prompt : “  “

Name: Knowledge Agent


In [36]:
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

In [37]:
model = ChatOllama(model="llama3.1:latest")

In [38]:
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

In [53]:
orchestrator = create_react_agent(
    model,
    [create_handoff_tool(agent_name="Knowledge Agent", description="Transfer when knowledge is needed, it helps to retrieve knowledge from the web using websearcher.")],
    prompt="""
        You are a Task Planner Agent responsible for coordinating a multi-agent system to analyze legal documents for discrepancies and compliance. Your job is to plan and delegate tasks to specialized agents, track task completion, and dynamically adapt the plan based on the current system state.

        You are aware of the capabilities of the agents and can query or instruct them based on the task at hand. After every task execution, you should validate whether the task was completed successfully. If a task fails or the output is insufficient, you should modify the workflow, reassign the task, or create additional subtasks.

        YOu have access to the following agents/tools:
        - Knowledge Agent: Transfer when knowledge is needed, it helps to retrieve knowledge from the web using websearcher.
        - Add: Add two numbers

        YOU ARE SUPPOSED TO PLAN WHAT AGENTS AND TOOLS TO CALL AND IN WHAT ORDER.
    """,
    name="Orchestrator",
)

In [54]:
knowledge = create_react_agent(
    model,
    [create_handoff_tool(agent_name="Orchestrator", description="Transfer to Orchestrator agent when knowledge is retrieved and pass a summary back to it.")],
    prompt="""
    Always return "hello world" as the response to the Orchestrator.
    """,
    name="Knowledge Agent",
)

In [55]:
checkpointer = InMemorySaver()
workflow = create_swarm(
    [orchestrator, knowledge],
    default_active_agent="Orchestrator"
)
app = workflow.compile(checkpointer=checkpointer)

In [56]:
config = {"configurable": {"thread_id": "1"}}
turn_1 = app.invoke(
    {"messages": [{"role": "user", "content": "Ask the Knowledge Agent to retrieve information and add the number of letters in the response."}]},
    config,
)
print(turn_1)


{'messages': [HumanMessage(content='Ask the Knowledge Agent to retrieve information and add the number of letters in the response.', additional_kwargs={}, response_metadata={}, id='d3bad1e8-a546-4981-b140-0a67642facb6'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-04-05T20:38:20.563282Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6350521917, 'load_duration': 41311667, 'prompt_eval_count': 360, 'prompt_eval_duration': 5278162000, 'eval_count': 30, 'eval_duration': 1012280375, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, name='Orchestrator', id='run-f0500f11-62d4-40bc-bfc4-26dbcb810f81-0', tool_calls=[{'name': 'transfer_to_knowledge_agent', 'args': {'question': 'What is the number of letters in the phrase \\'}, 'id': 'af61ebaf-05ac-44e0-a9d3-785f80fe5231', 'type': 'tool_call'}], usage_metadata={'input_tokens': 360, 'output_tokens': 30, 'total_tokens': 390}), ToolMessage(con

{'messages': [HumanMessage(content='Ask the Knowledge Agent to retrieve information and add the number of letters in the response.', additional_kwargs={}, response_metadata={}, id='d3bad1e8-a546-4981-b140-0a67642facb6'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-04-05T20:38:20.563282Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6350521917, 'load_duration': 41311667, 'prompt_eval_count': 360, 'prompt_eval_duration': 5278162000, 'eval_count': 30, 'eval_duration': 1012280375, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, name='Orchestrator', id='run-f0500f11-62d4-40bc-bfc4-26dbcb810f81-0', tool_calls=[{'name': 'transfer_to_knowledge_agent', 'args': {'question': 'What is the number of letters in the phrase \\'}, 'id': 'af61ebaf-05ac-44e0-a9d3-785f80fe5231', 'type': 'tool_call'}], usage_metadata={'input_tokens': 360, 'output_tokens': 30, 'total_tokens': 390}), ToolMessage(content='Successfully transferred to Knowledge Agent', name='transfer_to_knowledge_agent', id='43ee0968-488c-499b-b450-d3832fc23b7f', tool_call_id='af61ebaf-05ac-44e0-a9d3-785f80fe5231'), AIMessage(content='The Knowledge Agent has found the answer for you.\n\nKnowledge Agent Response:\n"hello world"\n\nSince there are 11 letters in the response, I can now format an answer to your original question.\n\nOriginal Question: What is the number of letters in the phrase "hello world"?\n\nAnswer: The phrase "hello world" contains 11 letters.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-04-05T20:38:23.527261Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2932665541, 'load_duration': 26135541, 'prompt_eval_count': 135, 'prompt_eval_duration': 617353167, 'eval_count': 69, 'eval_duration': 2280288292, 'message': Message(role='assistant', content='The Knowledge Agent has found the answer for you.\n\nKnowledge Agent Response:\n"hello world"\n\nSince there are 11 letters in the response, I can now format an answer to your original question.\n\nOriginal Question: What is the number of letters in the phrase "hello world"?\n\nAnswer: The phrase "hello world" contains 11 letters.', images=None, tool_calls=None)}, name='Knowledge Agent', id='run-9db84992-0c8e-49cc-ba77-385b220123e2-0', usage_metadata={'input_tokens': 135, 'output_tokens': 69, 'total_tokens': 204})], 'active_agent': 'Knowledge Agent'}

In [44]:
turn_2 = app.invoke(
    {"messages": [{"role": "user", "content": "what's 5 + 7?"}]},
    config,
)
print(turn_2["messages"][-1].content)

The answer is still 12.
